In [1]:
import os
import PyPDF2
import re

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file):
    text = ''
    with open(pdf_file, 'rb') as file:
        pdf_reader = PyPDF2.PdfFileReader(file)
        for page_num in range(pdf_reader.getNumPages()):
            page = pdf_reader.getPage(page_num)
            text += page.extractText()
    return text

# # Define regular expressions for job roles, skills, and education (customize these)
# job_role_pattern = r"(?i)(job\s*role|position|title):\s*([^\n]+)"
# skills_pattern = r"(?i)(Skill|skills|technical\s*skills):\s*([^\n]+)"
# education_pattern = r"(?i)(Education|Qualifications|Academic\s*background):\s*([^\n]+)"

# # Folder containing the PDF resumes
# pdf_folder = 'pdf'

# # Iterate through PDF files in the folder
# for filename in os.listdir(pdf_folder):
#     if filename.endswith('.pdf'):
#         pdf_file = os.path.join(pdf_folder, filename)
#         resume_text = extract_text_from_pdf(pdf_file)
        
#         # Extract job role
#         job_match = re.search(job_role_pattern, resume_text)
#         if job_match:
#             job_role = job_match.group(2)
#         else:
#             job_role = "Job Role Not Found"
        
#         # Extract skills
#         skills_match = re.search(skills_pattern, resume_text)
#         if skills_match:
#             skills = skills_match.group(2)
#         else:
#             skills = "Skills Not Found"
        
#         # Extract education
#         education_match = re.search(education_pattern, resume_text)
#         if education_match:
#             education = education_match.group(2)
#         else:
#             education = "Education Not Found"
        
#         # Print extracted information
#         print(f"File: {filename}")
#         print(f"Job Role: {job_role}")
#         print(f"Skills: {skills}")
#         print(f"Education: {education}")
#         print("\n")


In [2]:
import re,string
def clean_word(text):
    text = text.lower()    #we want to the convert our text into lower case
    text = re.sub('\[.*?\]', '', text)   #substitute these spacial character into empty space
    text = re.sub("\\W"," ",text)    #if any non-word character present it replaces that
    text = re.sub('https?://\S+|www\.\S+', '', text)   #it removes any google link or website,it replaced 
    text = re.sub('<.*?>+', '', text)   #these special cahracter also replaced with empty space..so these character will be removed
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)   #other than the above special character present in our text,it will replace
    text = re.sub('\n', '', text)   #if any \n means new line.it will replace
    text = re.sub('\w*\d\w*', '', text)    #if any number,non-word present in our text,it will replace
    #To remove the punctuations
    text = text.translate(str.maketrans(' ',' ',string.punctuation))
    #will consider only alphabets and numerics
    text = re.sub('[^a-zA-Z]',' ',text)  
    #will replace newline with space
    text = re.sub("\n"," ",text)
    # will split and join the words
    text=' '.join(text.split())
    return text

In [3]:
# Folder containing the PDF resumes
pdf_folder = 'pdf'
text1=[]
# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_file = os.path.join(pdf_folder, filename)
        resume_text = extract_text_from_pdf(pdf_file)
        resume_text1=clean_word(resume_text)
        text1.append(resume_text1)

In [4]:
len(text1)

1

In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Load pre-trained DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Read resumes and job descriptions from files (PDFs and CSVs)
resumes = text1  # List of resume texts (after PDF extraction and preprocessing)
job_descriptions = pd.read_csv('training_data.csv')['job_description'] # Replace with your CSV file

# Define a common maximum sequence length (you can adjust this)
max_seq_length = 512

# Tokenize and encode resumes and job descriptions with padding/truncation
resume_embeddings = []
job_description_embeddings = []

for resume_text in resumes:
    # Tokenize and encode resume text with padding/truncation
    inputs = tokenizer(resume_text, truncation=True, padding='max_length', max_length=max_seq_length, return_tensors="pt")
    with torch.no_grad():
        output = model(**inputs)
    # Average pooling over tokens
    resume_embedding = output.last_hidden_state.mean(dim=1)  # Average over tokens
    resume_embeddings.append(resume_embedding)

for job_description_text in job_descriptions:
    # Tokenize and encode job description text with padding/truncation
    inputs = tokenizer(job_description_text, truncation=True, padding='max_length', max_length=max_seq_length, return_tensors="pt")
    with torch.no_grad():
        output = model(**inputs)
    # Average pooling over tokens
    job_description_embedding = output.last_hidden_state.mean(dim=1)  # Average over tokens
    job_description_embeddings.append(job_description_embedding)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
resume_embeddings1 = [embedding.numpy().flatten() for embedding in resume_embeddings]
job_description_embeddings1 = [embedding.numpy().flatten() for embedding in job_description_embeddings]

In [7]:
# Calculate cosine similarity between each resume and each job description
similarity_matrix = cosine_similarity(resume_embeddings1, job_description_embeddings1)

In [8]:

# Display the similarity matrix
for i, resume_text in enumerate(resumes):
    for j, job_description_text in enumerate(job_descriptions):
        similarity_score = similarity_matrix[i][j]
        print(f"Resume {i+1} vs. Job Description {j+1}: Similarity Score = {similarity_score:.4f}")

Resume 1 vs. Job Description 1: Similarity Score = 0.9164
Resume 1 vs. Job Description 2: Similarity Score = 0.7631
Resume 1 vs. Job Description 3: Similarity Score = 0.9186
Resume 1 vs. Job Description 4: Similarity Score = 0.9049
Resume 1 vs. Job Description 5: Similarity Score = 0.9311
Resume 1 vs. Job Description 6: Similarity Score = 0.7966
Resume 1 vs. Job Description 7: Similarity Score = 0.9404
Resume 1 vs. Job Description 8: Similarity Score = 0.9273
Resume 1 vs. Job Description 9: Similarity Score = 0.9342
Resume 1 vs. Job Description 10: Similarity Score = 0.9438
Resume 1 vs. Job Description 11: Similarity Score = 0.9439
Resume 1 vs. Job Description 12: Similarity Score = 0.9258
Resume 1 vs. Job Description 13: Similarity Score = 0.9226
Resume 1 vs. Job Description 14: Similarity Score = 0.7545
Resume 1 vs. Job Description 15: Similarity Score = 0.9287
Resume 1 vs. Job Description 16: Similarity Score = 0.9306
Resume 1 vs. Job Description 17: Similarity Score = 0.8898
Resume

In [9]:
import pandas as pd

# Initialize an empty list to store similarity data
similarity_data = []

# Loop through resumes and job descriptions to collect data
for i, resume_text in enumerate(resumes):
    for j, job_description_text in enumerate(job_descriptions):
        similarity_score = similarity_matrix[i][j]
        data_entry = {
            'Resume Index': i,
            'Job Description Index': j ,
            'Similarity Score': similarity_score
        }
        similarity_data.append(data_entry)

# Create a DataFrame from the collected data
similarity_df = pd.DataFrame(similarity_data)


In [10]:
similarity_df

,Resume Index,Job Description Index,Similarity Score
0,0,0,0.916418
1,0,1,0.763070
2,0,2,0.918615
3,0,3,0.904852
4,0,4,0.931105
...,...,...,...
848,0,848,0.794055
849,0,849,0.935959
850,0,850,0.772082
851,0,851,0.897777


In [11]:
similarity_df = similarity_df.sort_values(by='Similarity Score', ascending=False)

In [12]:
similarity_df.head(10)

,Resume Index,Job Description Index,Similarity Score
73,0,73,0.958600
676,0,676,0.957516
140,0,140,0.952645
767,0,767,0.951942
63,0,63,0.951040
28,0,28,0.950584
163,0,163,0.949776
815,0,815,0.948868
312,0,312,0.948769
181,0,181,0.948727


In [25]:
resumes[2]

'vijay jawre cell no email gmail com objectives to obtain challenging position in server administration which will provide growth and learning opportunities permits to apply my experience quality of team work while completing challeging tasks assigned summary of experience years of it experience as a desktop admin and remote support engineer and responsible for managing and supporting desktops laptops active directory server administration and hardware maintainance implementation of active directory antivirus server troubleshooting windows directory linux server problems installation and operating windows macintosh and linux operating system installtion and support activities of lan and wan troubleshoot lan and wan network with various tools creating and mainting email profile users with various client softwares using remote control software tools logmein team viewer ammyy to provide fault resolution and diagnosis performing health checks on the server managing and checking for hardwar

In [26]:
job_descriptions[560]

'sr sap basis administrator\n\nprovide administration and technical support for all of j crews sap systems this job covers all aspects of sap basis administration including system installations and upgrades problem analysis and resolution database management client copies system refreshes sap instance configuration and performance tuning for a complex system landscape\n\nduties and responsibilities\n working at a technical level with an understanding of sap its underlying database operating system and hardware platform\n interact with development teams configuration teams various technical support teams and business stakeholders to optimize and maintain the overall sap reliability availability and performance\n the position has a  oncall commitment and availability for evening and weekend working is required\n provide installation configuration integration upgrade and testing services for new projects and ongoing maintenance for sap environments in a vm environment\n analyze diagnose a

In [31]:
job_descriptions[472]

'performs application onboarding for new third party and unfcu developed applications as well as complex upgrades for existing applications to ensure conformance with unfcu and industry guidelines for business and technical requirements as well as maintain high levels of security and availability of applications performs complex analysis on application issues upon escalation from lower level support teams and focuses on the ongoing capacity planning maintenance procedures and overall improvement of the application platform to deliver highquality and costeffective services to meet both short and longterm business requirements\n\nthis position is expected to be hybrid due to the nature of our business in new york city please note that unfcu will require that you show current proof of covid vaccination upon acceptance of employment\n\nnyc salary range     annually compensation is commensurate to geographic location\n administer various third party and unfcu developed applications includin